steps https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data 
- [x] load (download) data and setup 
- [x] explore and analyse data
- [ ] pipeline setup - modular, multiple handlable, simple transformer's dpr based at start
- [ ] inference
- [ ] prepare submit and submit - log the score with approach notes

- [ ] second iter


idealog
- [ ] training gpt (gpt neo) on cloud just to predict these keywords (generative approach)
- [ ] first find out section for dataset mention by reverse search and then do the topic modelling on each of such to identify main keywords that hints about mention of data, use this keyword based search to shortlist section dataset mention, and then run a bert - like that predict exact span
- [ ] if simple search doesn't work do dpr reverse search



notes 
>A percentage of the public test set publications are drawn from the training set - not all datasets have been identified in train, so these unidentified datasets have been used as a portion of the public test labels. These should serve as guides for the difficult task of labeling the private test set.

### Loading Data

In [16]:
# !pip install kaggle

     |████████████████████████████████| 58 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 1.6 MB/s eta 0:00:011
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=231f55facaa08039e45f9c6d5bb0a91bf0730f952955fe536e64f3b118aabded
  Stored in directory: /home/vishal_pathak_quantiphi_com/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=9d88ad1fda7ae8a61d92b0cae7e21f95f5ad9ed4a5bc054fe0e1210258206e37
  Stored in directory: /home/vishal_pathak_quantiphi_com/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
Successfully built kaggle python-slugify
You should consider upgrading via the '/home/vishal_pathak_quantiphi_com/covid-env/bin/python -m pip install --upgrade pip' command.


In [17]:
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/

In [19]:
# !kaggle competitions download -c coleridgeinitiative-show-us-the-data

 97%|███████████████████████████████████████▌ | 201M/208M [00:02<00:00, 108MB/s]
100%|█████████████████████████████████████████| 208M/208M [00:02<00:00, 107MB/s]


In [20]:
# !chmod 600 ~/.kaggle/kaggle.json

In [21]:
# !unzip coleridgeinitiative-show-us-the-data.zip

Archive:  coleridgeinitiative-show-us-the-data.zip
  inflating: sample_submission.csv   
  inflating: test/2100032a-7c33-4bff-97ef-690822c43466.json  
  inflating: test/2f392438-e215-4169-bebf-21ac4ff253e1.json  
  inflating: test/3f316b38-1a24-45a9-8d8c-4e05a42257c6.json  
  inflating: test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json  
  inflating: train.csv               
  inflating: train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json  
  inflating: train/0008656f-0ba2-4632-8602-3017b44c2e90.json  
  inflating: train/000e04d6-d6ef-442f-b070-4309493221ba.json  
  inflating: train/000efc17-13d8-433d-8f62-a3932fe4f3b8.json  
  inflating: train/0010357a-6365-4e5f-b982-582e6d32c3ee.json  
  inflating: train/002203f0-1c57-4400-abc1-b783c4085743.json  
  inflating: train/00243b98-f868-45e4-9b83-0c346c7ecad5.json  
  inflating: train/00248da3-ac1d-48fa-a95e-cc88553f9583.json  
  inflating: train/0026563b-d5b3-417d-bd25-7656b97a044f.json  
  inflating: train/002b53ee-5b30-497f-a2fd-56885563918c.js

In [22]:
# !mkdir -p coleridge_data
# !mv *.csv coleridge_data/
# !mv train coleridge_data/
# !mv test coleridge_data/
# !mv coleridge*.zip coleridge_data/

In [1]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

### Explore and analyze data

In [2]:
import os, json, random, re
import pandas as pd
from tqdm import tqdm

In [3]:
data_f = 'coleridge_data/'
train_csv = pd.read_csv(data_f + 'train.csv')

print(train_csv.shape, 'train_csv shape')
train_csv.sample(20)

(19661, 5) train_csv shape


Id  \
8087   bc45f3eb-b7a8-4ccb-8bbc-30cc0a501fda   
5024   9c10b7f2-ffde-493a-bd31-232be541c3a1   
18096  efaa895c-11f4-4263-9ee2-7c977665982a   
9956   e15fd181-667e-4008-8904-98f7b15dd84e   
14744  73709540-a811-45b9-b1ba-2db98e8a7161   
18264  fe130674-fb1d-4e52-a263-650944297bee   
15405  07befb53-e0e0-428e-9a62-76dca830b610   
9615   773a390a-8b92-40a6-931d-c78d76d8f980   
17855  291ef31b-adcb-4f26-a4ac-d99c426fb046   
5711   42388877-de41-4be4-b115-215de10322f3   
18131  a13f2e15-044a-428f-baad-4b119298611a   
10569  5a3a84db-4405-4922-87bb-960e05aa81e1   
19655  922a5f2c-2d1c-46a7-a07a-acaf2222c0c6   
7804   a2b72c54-4016-432e-972a-abf85e4c597f   
5240   567c4469-17f6-46e6-b2c6-e07e847052f9   
4644   a4b476a3-a67d-4269-a3d3-59399f5cdffc   
9877   4847a4e1-7e35-4d9b-8165-57ca883c273c   
13393  2b81a985-97eb-46ba-b535-c8f31e249c0c   
18688  86a59b7e-f2d1-4da3-8188-3fb203927b05   
7666   784fad14-eccf-417a-ab34-f475180939c5   

                                               pub_title  \
8087                             Rationality is Gendered   
5024   A Multi-Marker Genetic Association Test Based ...   
18096  The effects of management practices on grassla...   
9956   Evidence for Ordering of Alzheimer Disease Bio...   
14744        Higher Education in Science and Engineering   
18264  Customized Agricultural Resource Management Su...   
15405  Gender Differences in Academic Career Paths of...   
9615   Relationship between the Montreal Cognitive As...   
17855  Exploring predictions of abundance from body m...   
5711   Grid-based stochastic search for hierarchical ...   
18131  The Species Abundances Distribution in a new p...   
10569  Brain beta-amyloid measures and magnetic reson...   
19655  3D U-Net for segmentation of COVID-19 associat...   
7804   Validity and reliability of student perception...   
5240   A signature pattern of cortical atrophy in dem...   
4644   HENA, heterogeneous network-based data set for...   
9877   Rate of Conversion from Prodromal Alzheimer's ...   
13393  High School Longitudinal Study of 2009 (HSLS:0...   
18688  Evidence for possible association of vitamin D...   
7666   Viewing How STEM Project-Based Learning Influe...   

                                           dataset_title  \
8087                         Survey of Earned Doctorates   
5024   Alzheimer's Disease Neuroimaging Initiative (A...   
18096          North American Breeding Bird Survey (BBS)   
9956   Alzheimer's Disease Neuroimaging Initiative (A...   
14744                     Survey of Doctorate Recipients   
18264            Agricultural Resource Management Survey   
15405                     Survey of Doctorate Recipients   
9615   Alzheimer's Disease Neuroimaging Initiative (A...   
17855          North American Breeding Bird Survey (BBS)   
5711   Alzheimer's Disease Neuroimaging Initiative (A...   
18131          North American Breeding Bird Survey (BBS)   
10569  Alzheimer's Disease Neuroimaging Initiative (A...   
19655  RSNA International COVID-19 Open Radiology Dat...   
7804   Trends in International Mathematics and Scienc...   
5240   Alzheimer's Disease Neuroimaging Initiative (A...   
4644   Alzheimer's Disease Neuroimaging Initiative (A...   
9877   Alzheimer's Disease Neuroimaging Initiative (A...   
13393                 Early Childhood Longitudinal Study   
18688                 Our World in Data COVID-19 dataset   
7666   Trends in International Mathematics and Scienc...   

                                           dataset_label  \
8087                     NSF Survey of Earned Doctorates   
5024                                                ADNI   
18096                North American Breeding Bird Survey   
9956   Alzheimer's Disease Neuroimaging Initiative (A...   
14744                     Survey of Doctorate Recipients   
18264            Agricultural Resource Management Survey   
15405                     Survey of Doctorate Recipients   
9615   Alzheimer's Disease Neuroimaging Initiative 

In [4]:
print(train_csv.Id.unique().shape, 'unique ids') # repeated IDs means documents have multiple dataset labels

most_repeated_ids = train_csv.Id.value_counts()[:10]

(14316,) unique ids


In [5]:
print(most_repeated_ids)
train_csv[train_csv.Id.isin(most_repeated_ids.index)].head()

170113f9-399c-489e-ab53-2faf5c64c5bc    22
430aa11c-0283-411b-8edc-08f5df3db258    12
dd6df078-2010-42fa-ad30-9c302d2ba55b    12
0cbde738-c072-47e7-92b5-b6ccb5661f2d    11
3d61cdd8-240e-4202-9837-db4263fd196a    10
c82e4146-9a3e-45bd-8bf8-4866140b7412    10
5509b730-b7d5-459f-aa6b-72d84a81e97f     9
c4e22d3d-a953-4525-9f4f-97af0132c276     9
4a2bd823-e642-4ad9-b8e6-796ae4db9734     9
b262c60b-5b5f-4765-93bd-fb88458b4d7f     9
Name: Id, dtype: int64


Id  \
12917  4a2bd823-e642-4ad9-b8e6-796ae4db9734   
12918  4a2bd823-e642-4ad9-b8e6-796ae4db9734   
12919  4a2bd823-e642-4ad9-b8e6-796ae4db9734   
12920  4a2bd823-e642-4ad9-b8e6-796ae4db9734   
12921  4a2bd823-e642-4ad9-b8e6-796ae4db9734   

                                               pub_title  \
12917  Programs and Plans of the National Center for ...   
12918  Programs and Plans of the National Center for ...   
12919  Programs and Plans of the National Center for ...   
12920  Programs and Plans of the National Center for ...   
12921  Programs and Plans of the National Center for ...   

                         dataset_title  \
12917  Beginning Postsecondary Student   
12918         Baccalaureate and Beyond   
12919      Survey of Earned Doctorates   
12920         Baccalaureate and Beyond   
12921     Education Longitudinal Study   

                                           dataset_label  \
12917  Beginning Postsecondary Students Longitudinal ...   
12918        Baccalaureate and Beyond Longitudinal Study   
12919                        Survey of Earned Doctorates   
12920                           Baccalaureate and Beyond   
12921                       Education Longitudinal Study   

                                           cleaned_label  
12917  beginning postsecondary students longitudinal ...  
12918        baccalaureate and beyond longitudinal study  
12919                        survey of earned doctorates  
12920                           baccalaureate and beyond  
12921                       education longitudinal study

In [6]:
# distribution of no of dataset mentioned per paper
train_csv.Id.value_counts().describe() # for most of the papers only one dataset is reffered

count    14316.000000
mean         1.373358
std          0.647486
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         22.000000
Name: Id, dtype: float64

In [7]:
print(train_csv.cleaned_label.unique().shape, 'unique_cleaned labels (# of datasets)')
print(train_csv.cleaned_label.value_counts()[:10], 'most reffered datasets, with freq')

train_csv.cleaned_label.value_counts().describe() # most of the top dataset are mentioned over 10 times, while each dataset is mentioned at least twice

(130,) unique_cleaned labels (# of datasets)
adni                                                     3673
alzheimer s disease neuroimaging initiative adni         2400
trends in international mathematics and science study    1163
baltimore longitudinal study of aging                    1156
early childhood longitudinal study                       1011
education longitudinal study                              676
census of agriculture                                     643
agricultural resource management survey                   623
national education longitudinal study                     550
rural urban continuum codes                               490
Name: cleaned_label, dtype: int64 most reffered datasets, with freq


count     130.000000
mean      151.238462
std       425.998595
min         1.000000
25%         2.000000
50%        12.000000
75%       106.500000
max      3673.000000
Name: cleaned_label, dtype: float64

In [8]:
files = os.listdir(data_f + 'train')
print(len(files), 'no of files in train')

14316 no of files in train


In [9]:
random.choices(files, k=4)

['4891f564-a821-4805-9340-5abb635297e2.json',
 '5f31d6ea-7f81-4627-87e3-a9e6256ac967.json',
 'd054003e-5871-4773-b915-e45d506a0594.json',
 'a12b1723-bb91-4962-afe6-0a581df104b2.json']

In [10]:
# seeing train_data file and it's csv side by side

In [11]:
def get_json(uid, split='train'):
    filepath = data_f + split + '/' + uid + '.json'
    with open(filepath, 'r') as f:
        return json.load(f)

few_train_rows = train_csv.sample(5)
ids = few_train_rows.Id.values.tolist()
files = {uid: get_json(uid) for uid in ids}
id_csv = lambda uid, data=train_csv: data[data.Id==uid].values.tolist()

In [12]:
ids

['f22b09dd-8080-428d-bae8-4fc2bd64e412',
 'af948fc8-87be-4679-b3eb-b1c76605dcf8',
 'b873e970-5982-4af0-ba06-d6f8da8f09f1',
 '6af0e379-6508-4d93-8b7e-06296f7476d4',
 '0e47b600-7c6a-4936-a03f-67426d143f0a']

In [13]:
id_csv(ids[2], few_train_rows)

[['b873e970-5982-4af0-ba06-d6f8da8f09f1',
  'Comparison of Wetland Change Detection Data ',
  'Coastal Change Analysis Program',
  'Coastal Change Analysis Program',
  'coastal change analysis program']]

In [59]:
files[ids[2]] # can be seen in the last paragraph.

[{'section_title': 'Abstract',
  'text': 'Background Biomarkers provide clinicians with a predictable model for the diagnosis, treatment, and follow-up of medical ailments. Psychiatry has lagged behind other areas of medicine in the identification of biomarkers for clinical diagnosis and treatment. In this review, we investigated the current state of neuroimaging as it pertains to biomarkers for psychosis. Methods We reviewed systematic reviews and meta-analyses of the structural (sMRI), functional (fMRI), diffusion tensor (DTI), positron emission tomography (PET), and spectroscopy (MRS) studies of subjects at risk or those with an established schizophrenic illness. Only articles reporting effect sizes and confidence intervals were included in an assessment of robustness.'},
 {'section_title': 'Introduction',
  'text': 'Recent decades have witnessed major advances in understanding of the neurobiological basis of mental functions in health and disease, as well as an expanding basis of g

In [60]:
# as mentioned goal is to identify even those datasets who are never mentioned in the training dataset, sort of NER for dataset mentions

In [14]:
# for more explaination lets sample documents who have many datasets mentioned
docs_many_data = most_repeated_ids.index.tolist()
files_many_data = {i: get_json(i) for i in docs_many_data}

In [15]:
i = docs_many_data[2]
print(f'id {i}\ndatasets {[j[-1] for j in id_csv(docs_many_data[2])]}')

id dd6df078-2010-42fa-ad30-9c302d2ba55b
datasets ['beginning postsecondary students longitudinal study', 'baccalaureate and beyond longitudinal study', 'survey of earned doctorates', 'baccalaureate and beyond', 'survey of graduate students and postdoctorates in science and engineering', 'education longitudinal study', 'early childhood longitudinal study', 'beginning postsecondary students', 'trends in international mathematics and science study', 'national education longitudinal study', 'school survey on crime and safety', 'high school longitudinal study']


In [13]:
join_text = lambda jload: '\n\n\t'.join([i['section_title'] + ': ' + i['text'] for i in jload])

In [17]:
joined_text = join_text(files_many_data[i])
old_j = join_text(files[ids[2]])
print(len(joined_text), len(old_j), 'len of this doc v normal doc')

296122 4100 len of this doc v normal doc


In [76]:
print(joined_text[:32964])

: NATIONAL SCIENCE FOUNDATION

National Science Board

STEVEN C. BEERING (Chair), President Emeritus, Purdue University, West Lafayette
KATHRYN D. SULLIVAN (Vice Chair), Director, Battelle Center for Mathematics and Science Education Policy, John Glenn School of Public Affairs, Ohio State University, Columbus
MARK R. ABBOTT Dean and Professor, College of Oceanic and Atmospheric Sciences, Oregon State University
DAN E. ARVIZU Director and Chief Executive, National Renewable Energy Laboratory, Golden, Colorado
BARRY C. BARISH Maxine and Ronald Linde Professor of Physics Emeritus and Director, LIGO Laboratory, California Institute of Technology
CAMILLA P. BENBOW Patricia and Rodes Hart Dean of Education and Human Development, Peabody College, Vanderbilt University
RAY M. BOWEN President Emeritus, Texas A&M University, College Station
JOHN T. BRUER President, The James S. McDonnell Foundation, St. Louis
G. WAYNE CLOUGH President, Georgia Institute of Technology
KELVIN K. DROEGEMEIER Assoc

In [77]:
print(old_j)

Abstract: Background Biomarkers provide clinicians with a predictable model for the diagnosis, treatment, and follow-up of medical ailments. Psychiatry has lagged behind other areas of medicine in the identification of biomarkers for clinical diagnosis and treatment. In this review, we investigated the current state of neuroimaging as it pertains to biomarkers for psychosis. Methods We reviewed systematic reviews and meta-analyses of the structural (sMRI), functional (fMRI), diffusion tensor (DTI), positron emission tomography (PET), and spectroscopy (MRS) studies of subjects at risk or those with an established schizophrenic illness. Only articles reporting effect sizes and confidence intervals were included in an assessment of robustness.

	Introduction: Recent decades have witnessed major advances in understanding of the neurobiological basis of mental functions in health and disease, as well as an expanding basis of genetic and environmental etiology of mental disorders. Such knowl

### Pipeline

#### Data tagging (prep)

In [18]:
def find_index(text, target, width=100, find_only=False, lowr=True):
    if lowr:
        text = clean_text(text)
        target = clean_text(target)
    
    if find_only:
        return target in text
    
    if target in text:
        start = text.index(target)
        print('Snapshot', text[start-100: start+len(target)+100])
    
    else:
        print('target not found in the text, searching keywords')
        words = target.split()
        if len(words) <= 1: print(f'only one word! skipping {target}')
        else:
            for w in words:
                find_index(text, w, width=20)

In [19]:
targets = ['beginning postsecondary students longitudinal study', 'survey of earned doctorates', 'survey of science and engineering research facilities', 'survey of doctorate recipients', 'survey of graduate students and postdoctorates in science and engineering', 'education longitudinal study', 'early childhood longitudinal study', 'beginning postsecondary students', 'trends in international mathematics and science study', 'survey of industrial research and development', 'nsf survey of industrial research and development', 'nsf survey of earned doctorates']
text = joined_text
for tar in targets:
    print(f'for {tar}')
    find_index(text, tar)
    print()

for beginning postsecondary students longitudinal study
Snapshot d and final follow up b b 08 18 is planned for 2018 further information on b b may be obtained from beginning postsecondary students longitudinal study the beginning postsecondary students longitudinal study bps provides information on persistence pro

for survey of earned doctorates
Snapshot o the survey further information on federal funds for research and development may be obtained from survey of earned doctorates the survey of earned doctorates sed has collected basic statistics from the universe of doctoral re

for survey of science and engineering research facilities
target not found in the text, searching keywords
Snapshot 00 students who completed their degrees in the 1999 2000 academic year the first and only follow up survey of this cohort was conducted in 2001 b b 2000 01 and focused on time to degree completion participa
Snapshot on concerning education and work experience after completing a bachelor s degree

In [20]:
# out of 500 randomly selected how many's target are found as it is in text
def is_in_text(uid, label, lowr=False):
    jfile = get_json(uid)
    text = join_text(jfile)
    return find_index(text, label, find_only=True, lowr=lowr)

(
    train_csv.sample(500).apply(lambda row: is_in_text(row['Id'], row['dataset_label']), axis=1).value_counts(),
    train_csv.sample(500).apply(lambda row: is_in_text(row['Id'], row['cleaned_label'], lowr=True), axis=1).value_counts())


(True     450
 False     50
 dtype: int64,
 True    500
 dtype: int64)

#### data prep pipe
- collect all the sections and mentions against it
- filter out only sections with dataset mentioned in it
- train a hugging face's ner transformer on this
- run to predict on remaining sections
- get to know new dataset names

- finally two set of submission, one obtained from model, other from string match (with new discovered datasets)

In [21]:
train_csv.columns

Index(['Id', 'pub_title', 'dataset_title', 'dataset_label', 'cleaned_label'], dtype='object')

In [84]:
from_csv = lambda ind, col, csv=train_csv: csv.loc[ind, col]
class Section:
    csv = train_csv
    def __init__(self, jdata, csv_ind):
        self.name = jdata['section_title']
        self.text = clean_text(jdata['text'])
        self.csv_ind = csv_ind
        self.target = self.clean_label()
        self.doc_id = self.Id()
        
    
    def clean_label(self):
        return from_csv(self.csv_ind, 'cleaned_label')
    
    def title(self):
        return from_csv(self.csv_ind, 'pub_title')
    
    def Id(self):
        return from_csv(self.csv_ind, 'Id', Section.csv)
    
    def dataset_title(self):
        return from_csv(self.csv_ind, 'dataset_title')
    
    def dataset_label(self):
        return from_csv(self.csv_ind, 'dataset_label')
    
    def isin(self):
        return self.target in self.text
    
    def __repr__(self):
        return f"Sec(ind={self.csv_ind}, name={self.name}, doc_id={self.doc_id}, dataset={self.target})"
    
# sections = [Section(jdata, csv_ind) for csv_ind in tqdm(train_csv.index[:500]) for jdata in get_json(from_csv(csv_ind, 'Id'))]

In [23]:
sections_target = [i for i in sections if i.isin()]
print(len(sections_target), len(sections), 'targetted section and section length')

725 8154 targetted section and section length


In [24]:
sections_target[4]

Sec(ind=1, name=Defining Dropouts, doc_id=2f26f645-3dec-485d-b68d-f013c9e05e60, dataset=national education longitudinal study)

In [25]:
sections_target[4].text

'there are a number of ways to define dropouts for the purposes of this analysis students were considered dropouts if they ever reported dropping out of high school or passing the ged exam if their high school transcripts showed that they had dropped out or passed the ged exam or if in the 1994 or 2000 follow up surveys they reported that they were working on an alternative credential or that they had not completed high school and were not working on an alternative credential generally a student absent from school for 4 consecutive weeks or more and not absent due to accident or illness was considered to have dropped out see ingels et al 1994 members of the 1988 eighth grade cohort were classified as dropouts after they had dropped out for the first time whether or not they 1 includes students who earned a high school diploma or alternative credential by the time of the nels third follow up in spring 1994 2 includes students who earned a high school diploma or alternative credential be

In [131]:
# !pip install --upgrade transformers 

In [26]:
import transformers
transformers.__version__

'4.4.2'

In [27]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [28]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


a = tokenizer("Hello, this is one sentence!")

In [29]:
s = sections_target[4]
text_to_tokenize = s.text
target_char_index = s.text.index(s.target)
target_span = (target_char_index, target_char_index + len(s.target))

In [30]:
target_span

(1491, 1528)

In [31]:
toke = tokenizer(s.text)

In [32]:
star = toke.char_to_token(1491)
end = toke.char_to_token(1529)

In [33]:
star, end

(285, 289)

In [34]:
s.text

'there are a number of ways to define dropouts for the purposes of this analysis students were considered dropouts if they ever reported dropping out of high school or passing the ged exam if their high school transcripts showed that they had dropped out or passed the ged exam or if in the 1994 or 2000 follow up surveys they reported that they were working on an alternative credential or that they had not completed high school and were not working on an alternative credential generally a student absent from school for 4 consecutive weeks or more and not absent due to accident or illness was considered to have dropped out see ingels et al 1994 members of the 1988 eighth grade cohort were classified as dropouts after they had dropped out for the first time whether or not they 1 includes students who earned a high school diploma or alternative credential by the time of the nels third follow up in spring 1994 2 includes students who earned a high school diploma or alternative credential be

In [35]:
toke

{'input_ids': [101, 2045, 2024, 1037, 2193, 1997, 3971, 2000, 9375, 4530, 12166, 2005, 1996, 5682, 1997, 2023, 4106, 2493, 2020, 2641, 4530, 12166, 2065, 2027, 2412, 2988, 7510, 2041, 1997, 2152, 2082, 2030, 4458, 1996, 16216, 2094, 11360, 2065, 2037, 2152, 2082, 24051, 2015, 3662, 2008, 2027, 2018, 3333, 2041, 2030, 2979, 1996, 16216, 2094, 11360, 2030, 2065, 1999, 1996, 2807, 2030, 2456, 3582, 2039, 12265, 2027, 2988, 2008, 2027, 2020, 2551, 2006, 2019, 4522, 13675, 14728, 19909, 2030, 2008, 2027, 2018, 2025, 2949, 2152, 2082, 1998, 2020, 2025, 2551, 2006, 2019, 4522, 13675, 14728, 19909, 3227, 1037, 3076, 9962, 2013, 2082, 2005, 1018, 5486, 3134, 2030, 2062, 1998, 2025, 9962, 2349, 2000, 4926, 2030, 7355, 2001, 2641, 2000, 2031, 3333, 2041, 2156, 13749, 9050, 3802, 2632, 2807, 2372, 1997, 1996, 2997, 5964, 3694, 2522, 27794, 2020, 6219, 2004, 4530, 12166, 2044, 2027, 2018, 3333, 2041, 2005, 1996, 2034, 2051, 3251, 2030, 2025, 2027, 1015, 2950, 2493, 2040, 3687, 1037, 2152, 2082, 982

In [36]:
len(toke['input_ids'])

329

In [37]:
tokenizer.decode(toke['input_ids'][285:289])

'national education longitudinal study'

In [38]:
ner_tags = dict(O=0, B=1, I=2)

In [39]:
def tag_data(text, target):    
    left, right, center = *(text.split(target)[:2]), target
    
    # if mulitple splits
#     splits = text.split(target)
#     sp = []
#     for i in splits[:-1]:
#         sp.append(i)
#         sp.append(target)
#     sp.append(splits[-1])
    
    
    filt = lambda x: [i for i in x if i!='']
    left, center, right = list(map(lambda x: filt(x.split()), [left, center, right]))
    # alternates
#     final_tags, final_tokens = [], []
#     splits = list(map(lambda x: filt(x.split()), sp))
#     for i in range(0, len(splits), 2):
#         final_tokens.extend(splits[i])
#         final_tags.extend(['O'] * len(splits[i]))
        
#         if i+1 != len(splits):
#             final_tokens.extend(splits[i+1])
#             final_tags.append(['B'] + ['I']*(len(splits[i+1])-1))
    
    
    final_tokens = left + center + right
    final_tags = ['O']*len(left) + ['B',] + ['I']*(len(center)-1) + ['O']*len(right)    
    return final_tokens, final_tags

def tag_section(s):
    return tag_data(s.text, s.target)


list(zip(*tag_data('So hello from the other side, At least I can say then I tried.', 'other side')))

[('So', 'O'),
 ('hello', 'O'),
 ('from', 'O'),
 ('the', 'O'),
 ('other', 'B'),
 ('side', 'I'),
 (',', 'O'),
 ('At', 'O'),
 ('least', 'O'),
 ('I', 'O'),
 ('can', 'O'),
 ('say', 'O'),
 ('then', 'O'),
 ('I', 'O'),
 ('tried.', 'O')]

In [40]:
def get_dataset_dict(sections):
    tagged_data = [tag_section(s) for s in sections]
    return dict(
        mapping = dict(
            id=list(range(len(tagged_data))),
            tokens=[i for i,j in tagged_data],
            ner_tags=[list(map(ner_tags.get, j)) for i,j in tagged_data]            
        )
    )

from datasets import Dataset, load_metric

data = Dataset.from_dict(**get_dataset_dict(sections_target))


In [41]:
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(examples["tokens"], padding='max_length', truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [42]:
tokenize_and_align(data[:2])

{'input_ids': [[101, 2023, 2817, 2109, 2951, 2013, 1996, 2120, 2495, 20134, 2817, 11265, 4877, 6070, 2000, 11628, 1996, 3896, 1997, 7037, 10316, 3454, 2005, 2152, 2082, 2493, 2006, 2267, 3014, 18759, 3672, 1996, 2817, 2036, 2988, 3251, 1996, 14670, 1997, 7037, 10316, 3454, 2020, 2367, 2005, 2034, 4245, 2267, 2493, 6431, 2493, 3005, 3008, 2018, 3230, 2012, 2560, 2070, 2267, 1999, 2804, 1037, 27024, 4106, 4311, 2006, 1996, 4254, 1997, 2367, 8310, 1997, 7037, 10316, 2607, 2635, 1998, 2267, 3014, 18759, 3672, 7037, 10316, 3454, 3749, 2267, 2504, 4083, 6322, 2005, 2152, 2082, 2493, 1996, 3454, 3749, 2267, 5352, 1998, 2030, 1996, 4495, 2000, 7796, 2267, 6495, 2005, 2493, 2096, 2145, 1999, 2152, 2082, 1996, 8830, 2177, 1999, 1996, 2817, 2001, 11539, 1997, 11265, 4877, 6818, 2040, 3230, 1037, 8466, 8586, 29067, 2854, 2082, 1998, 2040, 4194, 1999, 1037, 7037, 10316, 2565, 2096, 1999, 2152, 2082, 1050, 26839, 1996, 2817, 3166, 2109, 17678, 6132, 3012, 3556, 9844, 4725, 2000, 3443, 1037, 7831, 21

In [43]:
tokenized_data = data.map(tokenize_and_align, batched=True)

In [44]:
label_list = list(ner_tags.keys())

In [45]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [53]:
args = TrainingArguments(
    f"test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [54]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [55]:
# !pip install seqeval
metric = load_metric("seqeval")

In [120]:
labels = data[f"{task}_tags"]
metric.compute(predictions=labels, references=labels)

{'_': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 725},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [49]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [56]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [58]:
trainer.train()

TrainOutput(global_step=546, training_loss=0.016047373226410522, metrics={'train_runtime': 8186.2417, 'train_samples_per_second': 0.067, 'total_flos': 443425633459200.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 3398427, 'init_mem_cpu_peaked_delta': 89208, 'train_mem_cpu_alloc_delta': 474775, 'train_mem_cpu_peaked_delta': 94605776})

In [52]:
batch_size=4

In [62]:
!ls

bamnet_exp  coleridge_data  EmbedKGQA  mlruns  random.ipynb  runs  test-ner


In [63]:
test_csv = pd.read_csv(data_f + 'sample_submission.csv')
test_csv

Id  PredictionString
0  2100032a-7c33-4bff-97ef-690822c43466               NaN
1  2f392438-e215-4169-bebf-21ac4ff253e1               NaN
2  3f316b38-1a24-45a9-8d8c-4e05a42257c6               NaN
3  8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60               NaN

In [75]:
def prepare_datasets(csv):
    sections = [Section(jdata, csv_ind) for csv_ind in tqdm(csv.index) for jdata in get_json(from_csv(csv_ind, 'Id', csv))]
    data = Dataset.from_dict(**get_dataset_dict(sections_target))
    tokenized_data = data.map(tokenize_and_align, batched=True)
    return tokenized_data, sections   
    

In [76]:
test_tokenized, test_sections = prepare_datasets(test_csv)

100%|██████████| 4/4 [00:00<00:00, 33.74it/s]


In [104]:
test_tokenized

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'tokens'],
    num_rows: 118
})

In [106]:
test_tokenized.remove_columns(['labels', 'ner_tags'])

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'tokens'],
    num_rows: 118
})

In [ ]:
model()

In [108]:
trainer.compute_metrics = None

In [115]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# results = metric.compute(predictions=true_predictions, references=true_labels)
# results

In [110]:
# predictions = numpy.array([[label_list[p] for p in pred] for pred in predictions])

In [165]:
def decode_labels(data, lables, skip='[SEP]'):
    
    for ix in tqdm(range(len(data))):
        # ix = 4
        predicted = [(i, j) for i, j in enumerate(labels[ix]) if j in [1, 2]]
        s = [(data[ix]['input_ids'][i], j) for i, j in predicted]
        s = tokenizer.decode([i for i, j in s])
        if s not in skip: print(ix, s)

In [166]:
len(test_tokenized)

118

In [167]:
# a page of requirement of KG paper

In [168]:
random_ids = random.choices(range(0, 700), k=5)
data = [{'input_ids': tokenized_data['input_ids'][i]} for i in random_ids]
labels = [tokenized_data['labels'][i] for i in random_ids]

decode_labels(data, labels)

100%|██████████| 5/5 [00:00<00:00, 1007.28it/s]

0 slosh model
1 slosh model
2 noaa tidal station
3 alzheimers disease neuroimaging initiative
4 national education longitudinal study


3

In [155]:
random_ids

[153, 353, 18, 139, 454]

In [169]:
train_csv.iloc[random_ids]

Id  \
564  51505a9b-b5ce-4132-a5fb-ec6aaf69c1b5   
510  149cd856-d8b4-4783-8df4-26cfd6287eeb   
375  acedff21-5ea9-4cf0-b207-a421698e03ec   
681  76f6b7f5-2b56-4410-87f8-dbc2f5f8a7e7   
264  6fd43867-841f-4490-b3d1-f38eb60b4f08   

                                             pub_title  \
564                       Kidney regeneration in fish.   
510  Space geodetic determination of spatial variab...   
375  Effects of Control Power and Inceptor Sensitiv...   
681  Prevalence and Outcomes of Frailty in Korean E...   
264  A General Panel Model with Random and Fixed Ef...   

                                      dataset_title  \
564    Baltimore Longitudinal Study of Aging (BLSA)   
510                                 NOAA Tide Gauge   
375  Sea, Lake, and Overland Surges from Hurricanes   
681    Baltimore Longitudinal Study of Aging (BLSA)   
264           National Education Longitudinal Study   

                                     dataset_label  \
564   Baltimore Longitudinal Study of Aging (BLSA)   
510  NOAA National Water Level Observation Network   
375                                    SLOSH model   
681   Baltimore Longitudinal Study of Aging (BLSA)   
264          National Education Longitudinal Study   

                                     cleaned_label  
564    baltimore longitudinal study of aging blsa   
510  noaa national water level observation network  
375                                    slosh model  
681    baltimore longitudinal study of aging blsa   
264          national education longitudinal study

In [171]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForTokenClassification

In [172]:
trainer.save_model??

Signature: trainer.save_model(output_dir:Union[str, NoneType]=None)
Source:   
    def save_model(self, output_dir: Optional[str] = None):
        """
        Will save the model, so you can reload it using :obj:`from_pretrained()`.

        Will only save from the main process.
        """
        if is_torch_tpu_available():
            self._save_tpu(output_dir)
        elif (
            ShardedDDPOption.ZERO_DP_2 in self.args.sharded_ddp or ShardedDDPOption.ZERO_DP_3 in self.args.sharded_ddp
        ):
            state_dict = self.model.state_dict()
            if self.is_world_process_zero():
                self._save(output_dir, state_dict=state_dict)
        elif self.is_world_process_zero():
            self._save(output_dir)
File:      ~/covid-env/lib/python3.6/site-packages/transformers/trainer.py
Type:      method


Init signature:
Trainer(
    model:torch.nn.modules.module.Module=None,
    args:transformers.training_args.TrainingArguments=None,
    data_collator:Union[DataCollator, NoneType]=None,
    train_dataset:Union[torch.utils.data.dataset.Dataset, NoneType]=None,
    eval_dataset:Union[torch.utils.data.dataset.Dataset, NoneType]=None,
    tokenizer:Union[_ForwardRef('PreTrainedTokenizerBase'), NoneType]=None,
    model_init:Callable[[], transformers.modeling_utils.PreTrainedModel]=None,
    compute_metrics:Union[Callable[[transformers.trainer_utils.EvalPrediction], Dict], NoneType]=None,
    callbacks:Union[List[transformers.trainer_callback.TrainerCallback], NoneType]=None,
    optimizers:Tuple[torch.optim.optimizer.Optimizer, torch.optim.lr_scheduler.LambdaLR]=(None, None),
)
Source:        
class Trainer:
    """
    Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.

    Args:
        model (:class:`~transformers.PreTrainedModel` 

In [176]:
# !mkdir -p saved_models/coleridge/distill-bert-finetuned1
trainer.save_model('saved_models/coleridge/distill-bert-finetuned1')

In [177]:
!du -sh saved_models/

254M	saved_models/


In [178]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13329176     7779792      377920       25580     5171464     5148136
Swap:             0           0           0


In [2]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        4.2G        3.5G         24M        4.9G        8.1G
Swap:            0B          0B          0B


## Simple version

In [7]:
sample = train_csv.sample(2000)

In [41]:
json_ids = sample.Id.unique()
labels_for_ids = {i: sample[sample.Id==i]['cleaned_label'].values.tolist() for i in json_ids}

# print([(i, j) for i, j in labels_for_ids.items() if len(j)>2])
data_f = 'coleridge_data/'

def get_json(uid, split='train'):
    filepath = data_f + split + '/' + uid + '.json'
    with open(filepath, 'r') as f:
        return json.load(f)

join_text = lambda jload: [i['section_title'] + ': ' + i['text'] for i in jload]
def find_index(text, target, width=100, find_only=False, lowr=True):
    if lowr:
        text = clean_text(text)
        target = clean_text(target)
    
    if find_only:
        return target in text
    
    if target in text:
        start = text.index(target)
        end = start + len(target)
        return start, end + 1
        # print('Snapshot', text[start-100: start+len(target)+100])
    
    else:
        print('target not found in the text, searching keywords')
        words = target.split()
        if len(words) <= 1: print(f'only one word! skipping {target}')
        else:
            founds = []
            for w in words:
                founds.append(find_index(text, w, width=20))
            return founds
    


def get_indx(doc, labels):
    return sorted(
        [find_index(doc, label) for label in labels 
            if find_index(doc, label, find_only=True)],
        key=lambda x: x[0])

def split_and_tag(doc, indx):
    if indx == []:
        return doc.strip().split(), ['O']*len(doc.split())
    splits = []
    tokens = []
    split_points = [0, ] + list({j for i in indx for j in i}) + [len(doc.strip().split()), ]
   #split_points = list({j for i in indx for j in i})
    split_points.sort()
    
    
    for s, e in zip(split_points, split_points[1:]):
        sp = doc[s:e].strip().split()
        if (s, e) in indx:
            tokens.append(['B',] + ['I'] * (len(sp) - 1))
        else : tokens.extend(['O']*len(sp))
        splits.extend(sp)
    
    return splits, tokens

doc_meta = {}
def prepare_data(labels_for_ids):
    data = []
    for _id, labels in tqdm(labels_for_ids.items()):
        file = get_json(_id)
        docs = join_text(file)
        for i, doc in enumerate(docs):
            para_id = f'{_id}_{i}'
            if para_id not in doc_meta:
                doc_meta[para_id] = dict(
                    para_id = f'{_id}_{i}',
                    doc_id = _id,
                    paragraph_no = i,
                    text = doc,
                    labels = labels                
                )            
            
            
            inds = get_indx(doc, labels)
            if inds!= []:
                tokens, tags = split_and_tag(doc, inds)
                data.append(dict(
                    tokens = tokens, 
                    tags = tags,
                    para_id = para_id
                ))
    globals().update(locals())
    return data

In [42]:
data = prepare_data(dict(list(labels_for_ids.items())[:10]))

100%|██████████| 10/10 [00:00<00:00, 124.31it/s]


In [46]:
tokens

['2:',
 'Analyses',
 'of',
 'Baccalaureate',
 'and',
 'Beyond',
 '92/03',
 'data',
 'indicate',
 'that',
 'White',
 'an']

In [44]:
labels

['baccalaureate and beyond']

In [47]:
def json_load(path):
    with open(path, 'r') as f:
        return json.load(f)

data_f = 'lm-v-kg/'
wiki_lookup = json_load(data_f+'wiki_lookup.json')

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 200937378 (char 200937377)

In [53]:
with open('lm-v-kg/wiki_lookup.json') as f:
    s = json.load(f)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 200937378 (char 200937377)

In [54]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        8.1G        1.9G         32M        2.7G        4.2G
Swap:            0B          0B          0B


In [ ]:
"/home/vishal_pathak_quantiphi_com/dpr-covid/doc_qa_module/elasticsearch/api_v3.py"